In [33]:
!pip install catboost

In [34]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [35]:
train = pd.read_csv('./data/full_train.csv')
test = pd.read_csv('./data/full_test.csv')
data = pd.read_csv('./data/full.csv')

# Straightforward solutions
Let's assume, that the best method for each time series is the method with the highest number of best results.
The accuarcy we get from choosing that method for every time series and the MAE between true best error and this method error will be taken as a baseline.

In [36]:
def straightforward(raw_data, best_data, criteria, models):
    baseline = pd.DataFrame(index=criteria,
                            columns=['metric', 'model', 'accuracy', 'lost_rate', 'MAE'])
    assert np.array(models).shape == np.array(criteria).shape
    baseline['metric'] = criteria
    baseline['model'] = models

    total = len(best_data)
    for criterion, model in zip(criteria, models):
        # Calculate accuarcy
        tp = best_data[best_data[criterion+'-model']==model].count().iloc[0]
        
        # For remaining FP results caculate MAE w.r.t. designated criterion
        # Get all rows with the best model
        true_results = raw_data[raw_data['model_name']==model]
        n = 0; mae = 0; lost = 0
        # For each time series calculate discrepancy between best result and
        # predicted method
        for ts, result in zip(best_data['naming_orig'], best_data[criterion]):
            batch = true_results[true_results['naming_orig']==ts]
            try:
                true_value = batch.iloc[batch[criterion].argmin()][criterion]
                if true_value != result:
                    n = n + 1
                    mae = mae + np.abs(true_value - result)
            except:
                lost = lost + 1
            
        baseline.loc[criterion, 'MAE'] = mae / n
        baseline.loc[criterion, 'lost_rate'] = lost / total
        baseline.loc[criterion, 'accuracy'] = tp / total
        baseline.index = np.arange(len(criteria))
            
    return baseline

## Experiment I
We take the most frequent model from the **train** dataset and apply it to the whole **train** dataset as a prediction.

In [37]:
best_train_model = train['RMSSE-model'].mode().to_numpy()[0]
train_baseline = straightforward(data[data['split']=='validation'],
                                 train, ['RMSSE'], [best_train_model])
train_baseline

,metric,model,accuracy,lost_rate,MAE
0,RMSSE,TFTTuningObjective_gl,0.150402,0.12744,0.320506


## Experiment II
We take the most frequent model from the **test** dataset and apply it to the whole **test** dataset as a prediction.

In [38]:
best_test_model = test['RMSSE-model'].mode().to_numpy()[0]
test_baseline = straightforward(data[data['split']=='validation'],
                                 test, ['RMSSE'], [best_test_model])
test_baseline

,metric,model,accuracy,lost_rate,MAE
0,RMSSE,Prophet,0.214696,0.0,0.195196


## Experiment III
We take the most frequent model from the **train** dataset and apply it to the whole **test** dataset as a prediction.

In [39]:
cv_baseline = straightforward(data[data['split']=='test'],
                              test, ['RMSSE'], [best_train_model])
cv_baseline

,metric,model,accuracy,lost_rate,MAE
0,RMSSE,TFTTuningObjective_gl,0.097589,0.12744,0.300779


## Experiment IV

In [40]:
def validation2test(raw_data, val_data, test_data, criteria):
    result = pd.DataFrame(index=criteria,
                          columns=['metric', 'model', 'accuracy', 'lost_rate', 'MAE'])
    assert np.array(val_data).shape == np.array(test_data).shape
    result['metric'] = criteria
    result['model'] = 'N/A'
    
    total = len(val_data)
    for criterion in criteria:
        # Calculate accuarcy
        tp = val_data[val_data[criterion+'-model']==test_data[criterion+'-model']].count().iloc[0]
        
        # For remaining FP results caculate MAE w.r.t. designated criterion
        # Get all rows with the best model
        n = 0; mae = 0; lost = 0
        # For each time series calculate discrepancy between best result and
        # predicted method
        for ts, metric in zip(test_data['naming_orig'], test_data[criterion]):
            batch = raw_data[raw_data['naming_orig']==ts]
            try:
                true_value = batch.iloc[batch[criterion].argmin()][criterion]
                if true_value != metric:
                    n = n + 1
                    mae = mae + np.abs(true_value - result)
            except:
                lost = lost + 1
            
        result.loc[criterion, 'MAE'] = mae / n
        result.loc[criterion, 'lost_rate'] = lost / total
        result.loc[criterion, 'accuracy'] = tp / total
        result.index = np.arange(len(criteria))
            
    return

In [41]:
val2test = validation2test(data[data['split']=='validation'], 
                           train, test, ['RMSSE'])
val2test

,metric,model,accuracy,lost_rate,MAE
0,RMSSE,N/A,0.167623,0.0,0.19816


# Classifier solutions

In [104]:
train_data = pd.read_csv("./data/train_actual.csv")
# train_data.reset_index()
# train_data.set_index('naming_orig')
# train_data
train_data = train_data.set_index('naming_orig')

X = train_data.drop('RMSSE_model', axis=1)
y = train_data['RMSSE_model']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostClassifier(iterations=18,  depth=7, learning_rate = 0.3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy on test set: {accuracy:.4f}")

0:	learn: 2.8105740	total: 71.8ms	remaining: 1.22s
1:	learn: 2.6663137	total: 136ms	remaining: 1.09s
2:	learn: 2.5409175	total: 204ms	remaining: 1.02s
3:	learn: 2.4399706	total: 276ms	remaining: 967ms
4:	learn: 2.3339324	total: 348ms	remaining: 905ms
5:	learn: 2.2503682	total: 417ms	remaining: 834ms
6:	learn: 2.1868034	total: 486ms	remaining: 763ms
7:	learn: 2.1218856	total: 553ms	remaining: 692ms
8:	learn: 2.0614959	total: 647ms	remaining: 647ms
9:	learn: 2.0110989	total: 717ms	remaining: 573ms
10:	learn: 1.9618075	total: 782ms	remaining: 497ms
11:	learn: 1.9064416	total: 853ms	remaining: 427ms
12:	learn: 1.8683643	total: 920ms	remaining: 354ms
13:	learn: 1.8296889	total: 987ms	remaining: 282ms
14:	learn: 1.7971875	total: 1.05s	remaining: 211ms
15:	learn: 1.7691655	total: 1.12s	remaining: 140ms
16:	learn: 1.7428628	total: 1.19s	remaining: 70ms
17:	learn: 1.7055079	total: 1.26s	remaining: 0us
Model accuracy on test set: 0.2989


In [108]:
def index2name(dataset, y):
    for i in range(len(y)):
        index = y[i,0]
        y[i,0] = dataset.iloc[index,0]
    return y

def asshole(raw_data, val_data, test_data, criteria):
    result = pd.DataFrame(index=criteria,
                          columns=['metric', 'model', 'accuracy', 'lost_rate', 'MAE'])
    #assert np.array(val_data).shape == np.array(test_data).shape
    result['metric'] = criteria
    result['model'] = 'N/A'
    
    total = len(val_data)
    for criterion in criteria:
        # Calculate accuarcy
        #tp = val_data[val_data[criterion+'-model']==test_data[criterion+'-model']].count().iloc[0]
        
        # For remaining FP results caculate MAE w.r.t. designated criterion
        # Get all rows with the best model
        n = 0; mae = 0; lost = 0
        # For each time series calculate discrepancy between best result and
        # predicted method
        for ts, metric in zip(test_data['naming_orig'], test_data[criterion]):
            batch = raw_data[raw_data['naming_orig']==ts]
            try:
                true_value = batch.iloc[batch[criterion].argmin()][criterion]
                if true_value != metric:
                    n = n + 1
                    mae = mae + np.abs(true_value - result)
            except:
                lost = lost + 1
            
        result.loc[criterion, 'MAE'] = mae / n
        result.loc[criterion, 'lost_rate'] = lost / total
        #result.loc[criterion, 'accuracy'] = tp / total
        result.index = np.arange(len(criteria))
            
    return

In [109]:
new_df = y_test.reset_index(drop=False)
new_df['RMSSE_model'] = y_pred[:,0]
new_df

,naming_orig,RMSSE_model
0,mipt_alpha_354,LightAutoML_mult_gl
1,danish_atm_daily_6,TFTTuningObjective_gl
2,mipt_alpha_447,LightAutoML
3,danish_atm_daily_60,TFTTuningObjective_gl
4,nn5_86,LightAutoML
...,...,...
169,danish_atm_daily_73,TFTTuningObjective_gl
170,mipt_alpha_503,LightAutoML_mult_gl
171,mipt_alpha_315,Prophet
172,mipt_alpha_490,CatBoostAutoRegressivePipelineEtna_horizonlags


In [110]:
exp4 = asshole(data[data['split']=='validation'], 
                           train_data, new_df, ['RMSSE'])
exp4

KeyError: 'RMSSE'

In [22]:
train_data = pd.read_csv("./data/train_actual.csv")
test_data = pd.read_csv("./data/test_actual.csv")

X_train = train_data.drop('RMSSE_model', axis=1)
y_train = train_data['RMSSE_model']

X_test = test_data.drop('RMSSE_model', axis=1)
y_true = test_data['RMSSE_model']

model = CatBoostClassifier(iterations=18,  depth=7, learning_rate = 0.3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_true, y_pred)
print(f"Model accuracy on test set: {accuracy:.4f}")
#model = XGBClassifier(n_estimators=7, max_depth=2, objective='multi:softmax', learning_rate = 0.1)

0:	learn: 2.7358668	total: 70.6ms	remaining: 1.2s
1:	learn: 2.5945471	total: 140ms	remaining: 1.12s
2:	learn: 2.4815120	total: 207ms	remaining: 1.03s
3:	learn: 2.3959946	total: 277ms	remaining: 970ms
4:	learn: 2.3102687	total: 348ms	remaining: 904ms
5:	learn: 2.2251430	total: 417ms	remaining: 835ms
6:	learn: 2.1600392	total: 487ms	remaining: 766ms
7:	learn: 2.0974175	total: 558ms	remaining: 698ms
8:	learn: 2.0402684	total: 630ms	remaining: 630ms
9:	learn: 2.0056483	total: 701ms	remaining: 560ms
10:	learn: 1.9657452	total: 778ms	remaining: 495ms
11:	learn: 1.9132387	total: 868ms	remaining: 434ms
12:	learn: 1.8759844	total: 937ms	remaining: 361ms
13:	learn: 1.8402355	total: 1.01s	remaining: 288ms
14:	learn: 1.8133323	total: 1.08s	remaining: 216ms
15:	learn: 1.7774503	total: 1.15s	remaining: 144ms
16:	learn: 1.7440936	total: 1.22s	remaining: 71.5ms
17:	learn: 1.7163924	total: 1.29s	remaining: 0us
Model accuracy on test set: 0.2440


In [6]:

# Load data from CSV files
train_data = pd.read_csv("train_actual.csv")
#test_data = pd.read_csv("test_actual.csv")

# Separate features (X) and target variable (y) for training data
X = train_data.drop('RMSSE_model', axis=1)
y = train_data['RMSSE_model']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier model
#model = RandomForestClassifier(n_estimators=300, random_state=42, max_depth = 7, min_samples_split = 5)
#model for usual case
#model = XGBClassifier(n_estimators=7, max_depth=2, objective='multi:softmax', learning_rate = 0.1)

#model for same target
#model = XGBClassifier(n_estimators=8, max_depth=10, objective='multi:softmax', learning_rate = 0.2, num_class = 20)

# model = CatBoostClassifier(iterations=18,  depth=7, learning_rate = 0.3)
# model = CatBoostClassifier()

# grid = {'learning_rate': [0.18, 0.24, 0.3, 0.35, 0.4],
#         'depth': [6, 7, 8, 9, 10],
#         'iterations': [16, 18, 20, 23, 25,],
#         #'subsample': [0.1, 0.3, 0.5, 0.7, 0.9],
#         'colsample_bylevel': [0.1, 0.3, 0.5, 0.7, 0.9],
#         'min_data_in_leaf': [7, 12, 20, 30, 40, 50]}

# grid_search_result = model.grid_search(grid, 
#                                        X=X_train, 
#                                        y=y_train, 
#                                        plot=True)

# le = LabelEncoder()
# y_train_encoded = le.fit_transform(y_train)

# Train the model


# Separate features (X_test) for testing data
# X_test = test_data.drop('RMSSE_model', axis=1)  # Assuming 'best_algorithm' exists in test data (for evaluation)

# Make predictions on the test set


# Evaluate model performance (optional, if 'best_algorithm' exists in test data)
# y_true = test_data['RMSSE_model']

# y_true_encoded = le.fit_transform(y_true)
# y_test_encoded = le.fit_transform(y_test)

#accuracy = accuracy_score(y_true_encoded, y_pred)


In [7]:
model = CatBoostClassifier(iterations=18,  depth=7, learning_rate = 0.3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy on test set: {accuracy:.4f}")

0:	learn: 2.8105740	total: 194ms	remaining: 3.31s
1:	learn: 2.6663137	total: 336ms	remaining: 2.69s
2:	learn: 2.5409175	total: 511ms	remaining: 2.55s
3:	learn: 2.4399706	total: 652ms	remaining: 2.28s
4:	learn: 2.3339324	total: 805ms	remaining: 2.09s
5:	learn: 2.2503682	total: 941ms	remaining: 1.88s
6:	learn: 2.1868034	total: 1.07s	remaining: 1.68s
7:	learn: 2.1218856	total: 1.2s	remaining: 1.5s
8:	learn: 2.0614959	total: 1.32s	remaining: 1.32s
9:	learn: 2.0110989	total: 1.47s	remaining: 1.18s
10:	learn: 1.9618075	total: 1.61s	remaining: 1.02s
11:	learn: 1.9064416	total: 1.75s	remaining: 876ms
12:	learn: 1.8683643	total: 1.89s	remaining: 725ms
13:	learn: 1.8296889	total: 2.02s	remaining: 578ms
14:	learn: 1.7971875	total: 2.15s	remaining: 431ms
15:	learn: 1.7691655	total: 2.29s	remaining: 286ms
16:	learn: 1.7428628	total: 2.43s	remaining: 143ms
17:	learn: 1.7055079	total: 2.56s	remaining: 0us
Model accuracy on test set: 0.2989


In [17]:
y_pred

array([['LightAutoML_mult_gl'],
       ['TFTTuningObjective_gl'],
       ['LightAutoML'],
       ['TFTTuningObjective_gl'],
       ['LightAutoML'],
       ['CatBoostAutoRegressivePipelineEtna_horizonlags'],
       ['TFTTuningObjective_gl'],
       ['CatBoostAutoRegressivePipelineEtna_horizonlags'],
       ['TFTTuningObjective_gl'],
       ['LightAutoML_mult_gl'],
       ['LightAutoML'],
       ['Prophet'],
       ['CatBoostAutoRegressivePipelineEtna_horizonlags'],
       ['LightAutoML_mult_gl'],
       ['LightAutoML_mult_gl'],
       ['Prophet'],
       ['Prophet'],
       ['LightAutoML'],
       ['CatBoostDirectPipelineEtna_horizonlags'],
       ['LightAutoML_mult_gl'],
       ['CatBoostAutoRegressivePipelineEtna_horizonlags'],
       ['CatBoostAutoRegressivePipelineEtna_horizonlags'],
       ['TFTTuningObjective_gl'],
       ['Prophet'],
       ['TFTTuningObjective_gl'],
       ['TFTTuningObjective_gl'],
       ['TFTTuningObjective_gl'],
       ['TFTTuningObjective_gl'],
       ['TFT

In [18]:
y_test

394    CatBoostAutoRegressivePipelineEtna_horizonlags_gl
66                                 TFTTuningObjective_gl
495                                          LightAutoML
67                                 TFTTuningObjective_gl
854                                              Prophet
                             ...                        
81                                   LightAutoML_mult_gl
558                                              Prophet
351       CatBoostAutoRegressivePipelineEtna_horizonlags
543             CatBoostAutoRegressivePipelineEtna_3lags
383                                  LightAutoML_mult_gl
Name: RMSSE_model, Length: 174, dtype: object